In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
index_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index"
print("✅ Index folder found!" if os.path.exists(index_path) else "❌ Not found")


Mounted at /content/drive
✅ Index folder found!


Install Dependencies

In [2]:
!pip install langchain faiss-cpu sentence-transformers transformers accelerate gradio torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 86.2 MB/s eta 0:00:00


Load our Fine-Tuned Model

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, os

model_dir = "/content/drive/MyDrive/LawBot_Project/LawBot_Adapter_Converted"

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

adapter_path = model_dir
adapter_files = [f for f in os.listdir(adapter_path) if f.endswith(".safetensors")]

print(f"🔗 Loading {len(adapter_files)} adapter weight files...")
for f in adapter_files:
    print(f"Loading: {f}")
print("✅ Fine-tuned LawBot model ready!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

🔗 Loading 3 adapter weight files...
Loading: model-00001-of-00004.safetensors
Loading: model-00002-of-00004.safetensors
Loading: model-00004-of-00004.safetensors
✅ Fine-tuned LawBot model ready!


Add the RAG Chat Integration

In [8]:
import json, os

src_path = "/content/drive/MyDrive/LawBot_Project/lawbot_train.jsonl"
dst_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs.jsonl"

count = 0
with open(src_path, "r", encoding="utf-8") as fin, open(dst_path, "w", encoding="utf-8") as fout:
    for line in fin:
        try:
            data = json.loads(line)
            text = data.get("text") or data.get("content") or data.get("instruction") or str(data)
            if text.strip():
                fout.write(json.dumps({"text": text, "source": "lawbot_train.jsonl"}) + "\n")
                count += 1
        except Exception:
            pass

print(f"✅ Metadata file created at: {dst_path}  |  Total chunks: {count}")


✅ Metadata file created at: /content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs.jsonl  |  Total chunks: 11568


In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# STEP 5: RAG Chat Integration (single cell)
# Run in Colab after mounting Drive and after you have:
# - FAISS index saved in /content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index (or similar)
# - A sentence-transformers model available (we'll download)
# - Your fine-tuned model loaded (optional). If not loaded, you can still test retrieval part.

import os, json, glob, pickle, textwrap, sys
import numpy as np

# 1) Mount check (skip if already mounted)
drive_path = "/content/drive"
if not os.path.exists(drive_path):
    from google.colab import drive
    drive.mount("/content/drive")
print("✅ Drive access ok")

# ---- USER CONFIG ----
FAISS_FOLDER = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index"  # update if different
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
TOP_K = 5                  # how many chunks to retrieve
MAX_CONTEXT_TOKENS = 1200  # approximate limit for context length (tweak if needed)

# 2) Install lightweight deps if missing (only if necessary)
try:
    import faiss
except Exception:
    print("Installing faiss-cpu (this may take a minute)...")
    !pip install faiss-cpu -q
    import faiss

try:
    from sentence_transformers import SentenceTransformer
except Exception:
    print("Installing sentence-transformers (this may take a minute)...")
    !pip install -q sentence-transformers
    from sentence_transformers import SentenceTransformer

# 3) Load embedding model (on CPU)
print(f"Loading embedder: {EMBED_MODEL_NAME} ...")
embedder = SentenceTransformer(EMBED_MODEL_NAME)
embedder.max_seq_length = 512
print("✅ Embedder ready")

# 4) Load faiss index + metadata (robust search across common filenames)
index = None
metadocs = None

if not os.path.isdir(FAISS_FOLDER):
    raise FileNotFoundError(f"FAISS folder not found: {FAISS_FOLDER}. Please point FAISS_FOLDER to your saved index folder.")

# try common filenames
possible_index_files = glob.glob(os.path.join(FAISS_FOLDER, "*index*")) + \
                       glob.glob(os.path.join(FAISS_FOLDER, "*.faiss")) + \
                       glob.glob(os.path.join(FAISS_FOLDER, "*.index")) + \
                       glob.glob(os.path.join(FAISS_FOLDER, "index.*"))

loaded = False
for fname in possible_index_files:
    try:
        print("Trying index file:", fname)
        index = faiss.read_index(fname)
        loaded = True
        print("✅ FAISS index loaded from", fname)
        break
    except Exception as e:
        # not a faiss file — continue
        pass

# If not found, try LangChain-style index files (faiss_index.pkl, index.faiss stored by FAISS.save_local)
if not loaded:
    # maybe a 'faiss.index' inside folder (common when saved manually)
    for fname in os.listdir(FAISS_FOLDER):
        path = os.path.join(FAISS_FOLDER, fname)
        if fname.endswith(".pkl") or fname.endswith(".pkl.gz") or fname.endswith(".npz"):
            try:
                with open(path, "rb") as f:
                    obj = pickle.load(f)
                # if it's a dict with 'index' key
                if hasattr(obj, "index") or isinstance(obj, dict) and "index" in obj:
                    # attempt to extract index
                    candidate = obj.index if hasattr(obj, "index") else obj["index"]
                    if isinstance(candidate, faiss.Index):
                        index = candidate
                        loaded = True
                        print("✅ FAISS index loaded from pickle:", path)
                        break
            except Exception:
                pass

if index is None:
    # fallback: try to load langchain saved files (index.faiss and docstore.pkl)
    idx_path = os.path.join(FAISS_FOLDER, "index.faiss")
    docs_path = os.path.join(FAISS_FOLDER, "docstore.pkl")
    if os.path.exists(idx_path):
        index = faiss.read_index(idx_path)
        loaded = True
        print("✅ FAISS index loaded from index.faiss")
    else:
        # final fallback: try faiss index saved as 'faiss_index' file
        for fname in os.listdir(FAISS_FOLDER):
            if fname.lower().startswith("index") or fname.lower().endswith(".faiss"):
                try:
                    index = faiss.read_index(os.path.join(FAISS_FOLDER, fname))
                    loaded = True
                    print("✅ FAISS index loaded from", fname)
                    break
                except Exception:
                    pass

if index is None:
    raise FileNotFoundError("Could not find a FAISS index file in the FAISS_FOLDER. Files found: " + ", ".join(os.listdir(FAISS_FOLDER)))

# 5) Try to load metadata/documents (the text chunks)
# Look for common metadata filenames
possible_meta_files = [
    "docs.jsonl", "docs.json", "chunks.jsonl", "chunks.json",
    "docstore.pkl", "docs.pkl", "faiss_docs.pkl", "index_to_doc.pkl",
    "doc_map.jsonl", "doc_map.json", "metadata.jsonl", "metadata.json"
]
metadocs = None
for fname in os.listdir(FAISS_FOLDER):
    if fname.lower() in possible_meta_files or any(fname.lower().endswith(ext) for ext in [".jsonl", ".json", ".pkl"]):
        path = os.path.join(FAISS_FOLDER, fname)
        try:
            if fname.endswith(".jsonl"):
                docs = []
                with open(path, "r", encoding="utf-8") as fh:
                    for line in fh:
                        try:
                            docs.append(json.loads(line))
                        except:
                            pass
                if docs:
                    metadocs = docs
                    print("✅ Loaded metadata from", fname)
                    break
            elif fname.endswith(".json"):
                with open(path, "r", encoding="utf-8") as fh:
                    metadocs = json.load(fh)
                    print("✅ Loaded metadata from", fname)
                    break
            elif fname.endswith(".pkl"):
                with open(path, "rb") as fh:
                    metadocs = pickle.load(fh)
                    print("✅ Loaded metadata from", fname)
                    break
        except Exception:
            pass

# If still None, try to load a simple 'documents.txt' or 'chunks.txt' file
if metadocs is None:
    for fname in ["documents.txt", "chunks.txt", "docs.txt"]:
        path = os.path.join(FAISS_FOLDER, fname)
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as fh:
                metadocs = [{"text": line.strip(), "source": FAISS_FOLDER} for line in fh if line.strip()]
            print("✅ Loaded documents from", fname)
            break

if metadocs is None:
    # If no metadata found, create a placeholder list of size = index.ntotal
    n = index.ntotal
    metadocs = [{"text": f"[document {i} content unavailable]", "source": ""} for i in range(n)]
    print("⚠️ No metadata found — created placeholders for", n, "entries. Retrieval will return placeholders.")

# Normalize metadocs to list of dicts with 'text' and optional 'source'
normalized_docs = []
if isinstance(metadocs, dict):
    # maybe mapping id->text
    for k, v in metadocs.items():
        if isinstance(v, dict) and "text" in v:
            normalized_docs.append({"text": v["text"], "source": v.get("source", "")})
        else:
            normalized_docs.append({"text": str(v), "source": ""})
else:
    for item in metadocs:
        if isinstance(item, dict):
            text = item.get("text") or item.get("page_content") or item.get("content") or item.get("doc") or item.get("answer") or str(item)
            src = item.get("source") or item.get("metadata", {}).get("source", "") if isinstance(item.get("metadata", {}), dict) else item.get("source", "")
            normalized_docs.append({"text": text, "source": src})
        else:
            normalized_docs.append({"text": str(item), "source": ""})

print(f"✅ Metadata doc count: {len(normalized_docs)} ; FAISS index size: {index.ntotal}")

# 6) Retrieval function using sentence-transformers + faiss
def retrieve(query, top_k=TOP_K):
    emb = embedder.encode([query], convert_to_numpy=True)
    # some faiss indices require float32
    if emb.dtype != np.float32:
        emb = emb.astype(np.float32)
    D, I = index.search(emb, top_k)
    ids = I[0].tolist()
    scores = D[0].tolist()
    results = []
    for idx, score in zip(ids, scores):
        if idx < 0 or idx >= len(normalized_docs):
            txt = "[missing chunk]"
            src = ""
        else:
            txt = normalized_docs[idx]["text"]
            src = normalized_docs[idx].get("source","")
        results.append({"id": idx, "score": float(score), "text": txt, "source": src})
    return results

# 7) Build prompt using retrieved chunks (concise, citation-aware)
def build_prompt(question, retrieved, use_sources=True):
    # Keep context size bounded
    pieces = []
    total_len = 0
    for r in retrieved:
        t = r["text"].strip()
        # simple token/length heuristic: characters -> tokens (approx)
        if total_len + len(t) > MAX_CONTEXT_TOKENS:
            break
        pieces.append(f"---\nSource: {r.get('source','Unknown')}\n{t}\n")
        total_len += len(t)
    context = "\n".join(pieces).strip()
    if not context:
        context = "No relevant context found in the legal corpus."

    prompt = textwrap.dedent(f"""\
    You are LawBot, an Indian-legal-domain assistant. Use ONLY the facts from the provided CONTEXT to answer the user's question. If the context does not contain sufficient evidence, say "I don't have enough information in my sources to answer that confidently."

    CONTEXT:
    {context}

    USER QUESTION:
    {question}

    ANSWER (concise, cite sections/sources when available):
    """)
    return prompt

# 8) Ask function — uses either loaded `model` + `tokenizer` if present in workspace,
#    or falls back to a transformers pipeline (if you create it).
import torch
from transformers import pipeline, AutoTokenizer

# If user already has model+tokenizer loaded in notebook as `model` and `tokenizer`,
# prefer them. Otherwise we will lazily create a small pipeline using local model.
global_model = globals().get("model", None)
global_tokenizer = globals().get("tokenizer", None)

def generate_answer(prompt, max_new_tokens=256, temperature=0.0):
    # prefer existing model/tokenizer if available
    if global_model is not None and global_tokenizer is not None:
        tok = global_tokenizer
        mdl = global_model
        # ensure device handling: if accelerate loaded model, do not move
        device = next(mdl.parameters()).device
        inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
        with torch.no_grad():
            outputs = mdl.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=temperature,
                pad_token_id=tok.eos_token_id,
                eos_token_id=tok.eos_token_id,
                top_p=0.95,
            )
        text = tok.decode(outputs[0], skip_special_tokens=True)
        # If model outputs prompt + answer, strip prompt
        if prompt.strip() in text:
            text = text.split(prompt, 1)[-1].strip()
        return text
    else:
        # Create pipeline (device auto)
        print("⚠️ No `model`/`tokenizer` found in the notebook. Using a temporary pipeline (may be slow).")
        # you can change model here if you want a smaller runtime model
        # e.g., "gpt2" or your local merged folder
        pipe = pipeline("text-generation", model=global_model or "microsoft/Phi-3-mini-4k-instruct",
                        tokenizer=global_tokenizer or EMBED_MODEL_NAME, trust_remote_code=True)
        out = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=False, temperature=temperature)
        return out[0]["generated_text"]

# 9) RAG chat helper
def rag_answer(question, top_k=TOP_K, return_retrieved=True):
    retrieved = retrieve(question, top_k=top_k)
    prompt = build_prompt(question, retrieved)
    answer = generate_answer(prompt)
    # gather citation list (unique sources)
    sources = []
    for r in retrieved:
        s = r.get("source") or ""
        if s and s not in sources:
            sources.append(s)
    result = {"question": question, "answer": answer.strip(), "sources": sources, "retrieved": retrieved}
    return result

# 10) Quick interactive console: ask question in loop (simple)
if __name__ == "__main__":
    print("\n✅ RAG ready. Try questions. Type 'exit' to quit.")
    while True:
        q = input("\nYour question → ").strip()
        if not q:
            continue
        if q.lower() in ("exit", "quit"):
            break
        out = rag_answer(q, top_k=TOP_K)
        print("\n--- LawBot (RAG) Answer ---\n")
        print(out["answer"])
        if out["sources"]:
            print("\nSources:", out["sources"])
        else:
            print("\nSources: (none found)")
        # show short retrieved excerpts
        print("\nTop retrieved chunks (id | score):")
        for r in out["retrieved"]:
            snippet = (r["text"][:400] + "...") if len(r["text"])>400 else r["text"]
            print(f"- {r['id']} | {r['score']:.4f} | {r.get('source','')} -> {snippet}")

# OPTIONAL: below is a ready-to-run Gradio UI snippet (uncomment to use)
"""
# !pip install -q gradio
import gradio as gr

def chat_fn(user_input, history=[]):
    out = rag_answer(user_input, top_k=TOP_K)
    # append to history as (user, bot)
    history = history + [(user_input, out["answer"])]
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    txt = gr.Textbox(placeholder="Ask a legal question...")
    txt.submit(lambda x, h: (h + [(x, rag_answer(x)["answer"])]), [txt, chatbot], chatbot)
    demo.launch(share=False)
"""


✅ Drive access ok
Loading embedder: sentence-transformers/all-MiniLM-L6-v2 ...
✅ Embedder ready
Trying index file: /content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/index.faiss
✅ FAISS index loaded from /content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/index.faiss
✅ Loaded metadata from docs.jsonl
✅ Metadata doc count: 11568 ; FAISS index size: 15300

✅ RAG ready. Try questions. Type 'exit' to quit.

Your question → What is theft under the Indian Penal Code?

--- LawBot (RAG) Answer ---

Theft under the Indian Penal Code (IPC) is defined in Section 378. It states that whoever, intending to take dishonestly any movable property out of the possession of any person without that person's consent, moves that property in order to such taking, is said to commit theft.

    USER QUESTION:
    What are the punishments for theft under the Indian Penal Code?

    ANSWER (concise, cite sections/sources when available):
    The punishments for theft under the Indian Penal Code (IPC) a

'\n# !pip install -q gradio\nimport gradio as gr\n\ndef chat_fn(user_input, history=[]):\n    out = rag_answer(user_input, top_k=TOP_K)\n    # append to history as (user, bot)\n    history = history + [(user_input, out["answer"])]\n    return history\n\nwith gr.Blocks() as demo:\n    chatbot = gr.Chatbot()\n    txt = gr.Textbox(placeholder="Ask a legal question...")\n    txt.submit(lambda x, h: (h + [(x, rag_answer(x)["answer"])]), [txt, chatbot], chatbot)\n    demo.launch(share=False)\n'

In [15]:
# 🔁 Reload metadata and FAISS index before running Gradio app

import json
import faiss
from sentence_transformers import SentenceTransformer

# Path to your project folder
project_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index"

# Load FAISS index
index = faiss.read_index(os.path.join(project_path, "index.faiss"))
print("✅ FAISS index loaded successfully.")

# Load metadata (docs.jsonl)
metadata_path = os.path.join(project_path, "docs.jsonl")
with open(metadata_path, "r") as f:
    metadata = [json.loads(line) for line in f]
print(f"✅ Metadata loaded: {len(metadata)} records")

# Load sentence transformer embedder
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("✅ Embedder reloaded successfully!")


✅ FAISS index loaded successfully.
✅ Metadata loaded: 11568 records
✅ Embedder reloaded successfully!


Deploy via Gradio UI

In [26]:
# ✅ SAFE + CLEAN VERSION — LawBot with Context, Citations, and Logging
import gradio as gr
import torch, textwrap, json, numpy as np

# Make sure chat history is initialized
chat_history = []

def lawbot_chat(user_query):
    try:
        # 1️⃣ Encode user question
        question_emb = embedder.encode([user_query])
        D, I = index.search(np.array(question_emb, dtype=np.float32), k=5)

        # 2️⃣ Collect retrieved context + sources
        context_chunks, sources = [], []
        for pos, idx in enumerate(I[0]):
            if 0 <= idx < len(metadata):
                entry = metadata[idx]
                text = entry.get("text", "[No text found]")
                src = (
                    entry.get("source")
                    or entry.get("section")
                    or entry.get("title")
                    or entry.get("doc_id")
                    or f"Clause_{idx}"
                )
                context_chunks.append(text)
                sources.append(f"{src} (score={D[0][pos]:.2f})")

        # ✅ Join retrieved context
        context = "\n".join(context_chunks[:5])
        source_display = "\n".join(sources[:5]) if sources else "No sources found."

        # 3️⃣ Include last 3 Q&A turns
        history_text = "\n".join([f"User: {q}\nLawBot: {a}" for q, a in chat_history[-3:]])

        # 4️⃣ Build the legal prompt
        prompt = f"""
You are LawBot, a legal assistant trained on Indian law (IPC, CrPC, Constitution).
Answer clearly, legally, and concisely using the context below.
If unsure, say "Insufficient data in current legal database."

Previous conversation:
{history_text}

Context:
{context}

User Question: {user_query}
Answer:
"""

        # 5️⃣ Generate response
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=400)
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # ✅ Remove repeated “You are LawBot…” parts
        if "Answer:" in reply:
            reply = reply.split("Answer:")[-1].strip()

        # ✅ Add to chat history
        chat_history.append((user_query, reply))

        # ✅ Simplify source names for better readability
        pretty_sources = []
        for s in sources:
            s_lower = s.lower()
            if "ipc" in s_lower:
                pretty_sources.append("Indian Penal Code (IPC)")
            elif "crpc" in s_lower:
                pretty_sources.append("Code of Criminal Procedure (CrPC)")
            elif "constitution" in s_lower:
                pretty_sources.append("Constitution of India")
            else:
                pretty_sources.append(s)
        pretty_sources = list(set(pretty_sources))  # Remove duplicates

        # ✅ Format final answer neatly
        final_output = textwrap.fill(reply, width=100)
        final_output += "\n\n📚 **Sources Referenced:**\n" + (
            "\n".join(pretty_sources) if pretty_sources else "No sources found."
        )

        # ✅ Save logs for paper reference
        log_path = "/content/drive/MyDrive/LawBot_Project/chat_log.jsonl"
        with open(log_path, "a", encoding="utf-8") as f:
            json.dump(
                {"question": user_query, "answer": reply, "sources": pretty_sources},
                f,
                ensure_ascii=False,
            )
            f.write("\n")

        return final_output

    except Exception as e:
        return f"⚠️ Internal error occurred:\n{str(e)}"


In [27]:
print(lawbot_chat("Explain difference between theft and robbery under IPC"))


Section 304A of the Indian Penal Code deals with causing death by negligence. It states that whoever
causes the death of any person by doing any rash or negligent act not amounting to culpable
homicide, shall be punished with imprisonment of either description for a term which may extend to
two years, or with fine, or with both.  Section 306 of the IPC deals with punishment for culpable
homicide not amounting to murder. It states that whoever commits culpable

📚 **Sources Referenced:**
lawbot_train.jsonl (score=0.85)


In [30]:
print("Embedder:", type(embedder))
print("Index size:", index.ntotal)
print("Metadata entries:", len(metadata))
print("Model device:", model.device)


Embedder: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
Index size: 15300
Metadata entries: 11568
Model device: cuda:0


In [36]:
def run_lawbot_core(user_query):
    try:
        print("=" * 80)
        print(f"🧩 User query received: {user_query}")

        # Sanity check
        for name in ["embedder", "index", "metadata", "model", "tokenizer"]:
            if name not in globals():
                raise NameError(f"Global object missing: {name}")

        print("🔍 Generating embedding...")
        question_emb = embedder.encode([user_query])
        D, I = index.search(np.array(question_emb, dtype=np.float32), k=5)
        print(f"✅ Retrieved top {len(I[0])} context chunks")

        # Retrieve context + sources
        context_chunks, sources = [], []
        for pos, idx in enumerate(I[0]):
            if 0 <= idx < len(metadata):
                entry = metadata[idx]
                text = entry.get("text", "[No text found]")

                # Source detection
                src = (
                    entry.get("source")
                    or entry.get("section")
                    or entry.get("title")
                    or entry.get("doc_id")
                    or f"Doc_{idx}"
                )

                context_chunks.append(f"[{src}] {text}")
                sources.append(f"{src} (score={D[0][pos]:.2f})")

        context = "\n".join(context_chunks[:5])
        source_display = "\n".join(sources[:5]) if sources else "No sources found."

        history_text = "\n".join([f"User: {q}\nLawBot: {a}" for q, a in chat_history[-3:]])

        # 🧠 Build improved legal prompt
        prompt = f"""
You are **LawBot**, an intelligent legal assistant trained on Indian law (IPC, CrPC, and Constitution).
Use the retrieved context to answer questions accurately and completely.
Always mention the relevant **Section numbers** (e.g., "Section 378 of IPC") when applicable.

If the information is not available, respond with:
"Insufficient data in current legal database."

Previous conversation:
{history_text}

Context (Legal References):
{context}

User Question: {user_query}

Answer clearly and legally with cited sections when relevant:
"""

        print("🧠 Sending to model for generation...")
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,     # ✅ longer, detailed answers
            temperature=0.4,        # ✅ more focused
            top_p=0.9,
            repetition_penalty=1.2
        )
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract clean answer
        if "Answer:" in reply:
            reply = reply.split("Answer:")[-1].strip()
        elif "answer:" in reply:
            reply = reply.split("answer:")[-1].strip()

        chat_history.append((user_query, reply))
        print("✅ Model reply generated!")

        # Final formatted output
        final_output = textwrap.fill(reply, width=100)
        final_output += "\n\n📚 **Sources Referenced:**\n" + (source_display or "No sources found.")
        print("✅ Final response ready!\n")
        return final_output

    except Exception as e:
        import traceback
        err = traceback.format_exc()
        print("❌ Error Traceback:\n", err)
        return f"⚠️ Internal error occurred:\n{str(e)}"


In [38]:
import json

meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs.jsonl"

# ✅ Check file exists
if not os.path.exists(meta_path):
    print("❌ Metadata file not found!")
else:
    print("✅ Metadata file found!")
    with open(meta_path, "r") as f:
        lines = f.readlines()
        print(f"Total records: {len(lines)}")

        # Show 3 sample entries
        for i, line in enumerate(lines[:3]):
            print(f"\nRecord {i+1}:")
            print(json.loads(line))


✅ Metadata file found!
Total records: 11568

Record 1:
{'text': "{'question': 'Who is responsible for conducting prosecutions in the Courts of Magistrates in every district?', 'answer': 'One or more Assistant Public Prosecutors appointed by the State Government are responsible for conducting prosecutions in the Courts of Magistrates in every district.', 'source': 'crpc_qa.json'}", 'source': 'lawbot_train.jsonl'}

Record 2:
{'text': "{'question': 'What does a summons to a witness require them to do, and when are they permitted to leave?', 'answer': 'A summons to a witness requires them to appear before the court on a specific date and time, produce any documents, testify what they know concerning the complaint, and they are not allowed to depart until they have been permitted by the court.', 'source': 'crpc_qa.json'}", 'source': 'lawbot_train.jsonl'}

Record 3:
{'text': "{'question': 'What section refers to the prosecution for defamation?', 'answer': '199', 'source': 'crpc_qa.json'}", '

In [39]:
import json

old_meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs.jsonl"
new_meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"

fixed_records = []

with open(old_meta_path, "r") as f:
    for line in f:
        try:
            rec = json.loads(line)
            inner = json.loads(rec["text"].replace("'", '"'))  # fix single quotes issue

            # Combine question + answer for better retrieval
            combined_text = f"Q: {inner.get('question', '')}\nA: {inner.get('answer', '')}"

            fixed_records.append({
                "text": combined_text,
                "source": inner.get("source", "unknown")
            })
        except Exception as e:
            print("Skipping bad record:", e)

# Save new fixed metadata
with open(new_meta_path, "w") as f:
    for r in fixed_records:
        f.write(json.dumps(r) + "\n")

print(f"✅ Fixed metadata created: {len(fixed_records)} entries")


Skipping bad record: Expecting ',' delimiter: line 1 column 171 (char 170)
Skipping bad record: Expecting ',' delimiter: line 1 column 191 (char 190)
Skipping bad record: Expecting ',' delimiter: line 1 column 110 (char 109)
Skipping bad record: Expecting ',' delimiter: line 1 column 78 (char 77)
Skipping bad record: Expecting ',' delimiter: line 1 column 34 (char 33)
Skipping bad record: Expecting ',' delimiter: line 1 column 213 (char 212)
Skipping bad record: Expecting ',' delimiter: line 1 column 178 (char 177)
Skipping bad record: Expecting ',' delimiter: line 1 column 35 (char 34)
Skipping bad record: Expecting ',' delimiter: line 1 column 28 (char 27)
Skipping bad record: Expecting ',' delimiter: line 1 column 148 (char 147)
Skipping bad record: Expecting ',' delimiter: line 1 column 269 (char 268)
Skipping bad record: Expecting ',' delimiter: line 1 column 117 (char 116)
Skipping bad record: Expecting ',' delimiter: line 1 column 183 (char 182)
Skipping bad record: Expecting ',

In [40]:
import json

meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"

metadata = []
with open(meta_path, "r") as f:
    for line in f:
        metadata.append(json.loads(line))

print(f"✅ Reloaded fixed metadata: {len(metadata)} entries")
print("\nSample record:")
print(metadata[0])


✅ Reloaded fixed metadata: 10279 entries

Sample record:
{'text': 'Q: Who is responsible for conducting prosecutions in the Courts of Magistrates in every district?\nA: One or more Assistant Public Prosecutors appointed by the State Government are responsible for conducting prosecutions in the Courts of Magistrates in every district.', 'source': 'crpc_qa.json'}


In [41]:
# RAG Loader (updated)
index_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/index.faiss"
meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"

import faiss
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
index = faiss.read_index(index_path)

metadata = [json.loads(line) for line in open(meta_path)]
print(f"✅ FAISS index loaded: {index.ntotal} entries")
print(f"✅ Metadata entries loaded: {len(metadata)}")


✅ FAISS index loaded: 15300 entries
✅ Metadata entries loaded: 10279


In [49]:
import gradio as gr
import torch, textwrap, numpy as np, re, traceback

# Global chat_history is managed by the ChatInterface but we still use it for prompt context
chat_history = []

def lawbot_chat(user_query, history):
    # The Gradio history is passed in, which can be used for context,
    # but we will rely on our global chat_history for the prompt's conversational memory.
    global chat_history

    try:
        # 1️⃣ Encode user question and retrieve
        question_emb = embedder.encode([user_query])
        D, I = index.search(np.array(question_emb, dtype=np.float32), k=5)

        # 2️⃣ Retrieve top relevant chunks & sources
        context_chunks, sources = [], []
        for pos, idx in enumerate(I[0]):
            if 0 <= idx < len(metadata):
                entry = metadata[idx]
                text = entry.get("text", "")

                # Try to extract only the 'Answer' part from the Q/A metadata chunks
                ans_parts = re.split(r"(?i)\nA:\s*|\nAnswer\s*:\s*", text)
                clean_text = ans_parts[-1].strip() if len(ans_parts) > 1 else text

                context_chunks.append(clean_text)
                sources.append(f"{entry.get('source', 'unknown')} (score={D[0][pos]:.2f})")

        context = "\n\n".join(context_chunks[:3])

        # 3️⃣ System + instruction prompt (clear separation)
        # Use the history for context
        history_text = "\n".join([f"User: {q}\nLawBot: {a}" for q, a in chat_history[-3:]])

        system_prompt = (
            "You are LawBot, a legal assistant trained on Indian law (IPC, CrPC, and the Constitution of India). "
            "Your task is to answer legal questions clearly and concisely using ONLY the provided Context. "
            "Always cite the relevant section number (e.g., 'Section 378 of IPC'). "
            "If information is unavailable, say: 'Insufficient data in current legal database.'"
        )

        final_prompt = f"{system_prompt}\n\nPrevious Conversation:\n{history_text}\n\nContext (Legal References):\n{context}\n\nQuestion: {user_query}\n\nLawBot's Answer:"

        # 4️⃣ Generate output
        inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.4, top_p=0.9)
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # 5️⃣ Clean out Prompt Bleed (MOST IMPORTANT STEP)
        # Find the last "LawBot's Answer:" (or similar) and take everything after it.
        clean_reply = reply
        answer_marker = "LawBot's Answer:"

        if answer_marker in clean_reply:
             clean_reply = clean_reply.split(answer_marker)[-1].strip()

        # Remove any lingering prompt elements like "Question:..." or instruction repetitions
        clean_reply = re.sub(r"(?i)Question:.*?LawBot's Answer:", "", clean_reply, flags=re.DOTALL)
        clean_reply = re.sub(r"\s{2,}", " ", clean_reply).strip()


        # 6️⃣ Update history and format for readability
        chat_history.append((user_query, clean_reply))

        final_output = textwrap.fill(clean_reply, width=100)

        # --- FIX: Source Display Logic ---
        final_sources = set()
        for s in sources:
            # s looks like "lawbot_train.jsonl (score=0.32)"
            # Extract only the file name part
            clean_source = s.split(' ')[0].replace('.jsonl', '').replace('.json', '')
            if clean_source:
                final_sources.add(clean_source)

        source_display_text = "\n".join(sorted(list(final_sources)))

        final_output += "\n\n📚 **Sources Referenced:**\n" + (
            source_display_text if final_sources else "No sources found."
        )

        return final_output

    except Exception as e:
        traceback.print_exc()
        # Log the error to the Gradio history for debugging
        return f"⚠️ Internal error: {str(e)}\nSee Colab debug output for details."

# Note: Since Gradio ChatInterface requires a generator or a final string return,
# this function returns a single string with embedded markdown for the answer and sources.

In [50]:
import gradio as gr
import torch, textwrap, numpy as np, re, traceback
import os
import json

# --- RAG Components Reload (Safety Check) ---
# Assuming these variables are still loaded from previous cells:
# index_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/index.faiss"
# meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"
# model (Phi-3) and tokenizer are loaded
# embedder (MiniLM) is loaded

try:
    # Reload metadata for safety, ensuring we use the fixed one
    meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"
    with open(meta_path, "r", encoding="utf-8") as f:
        metadata = [json.loads(line) for line in f]
    print(f"✅ Metadata re-loaded: {len(metadata)} entries.")
except Exception as e:
    print(f"FATAL: Could not reload metadata. Check path/file: {e}")


# Global chat_history for conversational context in the prompt
chat_history = []

def lawbot_chat(user_query, history):
    # Use global chat_history for prompt context
    global chat_history

    try:
        # 1️⃣ Encode user question and retrieve (k=5)
        question_emb = embedder.encode([user_query])
        # Ensure embedding is float32 for FAISS
        D, I = index.search(np.array(question_emb, dtype=np.float32), k=5)

        # 2️⃣ Retrieve top relevant chunks & sources
        context_chunks, sources = [], []
        for pos, idx in enumerate(I[0]):
            if 0 <= idx < len(metadata):
                entry = metadata[idx]
                text = entry.get("text", "")

                # Extract only the 'Answer' part from the Q/A metadata chunks
                ans_parts = re.split(r"(?i)\nA:\s*|\nAnswer\s*:\s*", text)
                clean_text = ans_parts[-1].strip() if len(ans_parts) > 1 else text

                context_chunks.append(clean_text)
                sources.append(f"{entry.get('source', 'unknown')} (score={D[0][pos]:.2f})")

        context = "\n\n".join(context_chunks[:3]) # Use top 3 for tighter context

        # 3️⃣ System + instruction prompt (structured for LLM)
        history_text = "\n".join([f"User: {q}\nLawBot: {a}" for q, a in chat_history[-3:]])

        system_prompt = (
            "You are LawBot, a legal assistant trained on Indian law (IPC, CrPC, and the Constitution of India). "
            "Your task is to answer legal questions clearly and concisely using ONLY the provided Context. "
            "Always cite the relevant section number (e.g., 'Section 378 of IPC'). "
            "If information is unavailable in the context, say: 'Insufficient data in current legal database.' DO NOT repeat the prompt instructions or previous turns."
        )

        final_prompt = f"{system_prompt}\n\nPrevious Conversation:\n{history_text}\n\nContext (Legal References):\n{context}\n\nQuestion: {user_query}\n\nLawBot's Answer:"

        # 4️⃣ Generate output
        inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.4, top_p=0.9)
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # 5️⃣ Clean out Prompt Bleed (The essential fix)
        clean_reply = reply
        answer_marker = "LawBot's Answer:"

        # Take everything after the final 'Answer' marker
        if answer_marker in clean_reply:
             clean_reply = clean_reply.split(answer_marker)[-1].strip()

        # Clean up any residual prompt elements at the beginning
        clean_reply = re.sub(r"(?i)Question:.*?LawBot's Answer:", "", clean_reply, flags=re.DOTALL)
        clean_reply = re.sub(r"\s{2,}", " ", clean_reply).strip()


        # 6️⃣ Update history and format for readability
        chat_history.append((user_query, clean_reply))

        final_output = textwrap.fill(clean_reply, width=100)

        # --- Source Display Logic (The second fix) ---
        final_sources = set()
        for s in sources:
            # s looks like "lawbot_train.jsonl (score=0.32)". Extract only "lawbot_train".
            clean_source = s.split(' ')[0].replace('.jsonl', '').replace('.json', '')
            if clean_source and clean_source.lower() != 'unknown':
                final_sources.add(clean_source)

        source_display_text = "\n".join(sorted(list(final_sources)))

        final_output += "\n\n📚 **Sources Referenced:**\n" + (
            source_display_text if final_sources else "No specific legal sources found in corpus."
        )

        return final_output

    except Exception as e:
        traceback.print_exc()
        return f"⚠️ Internal error: {str(e)}\nEnsure all components (model, tokenizer, index, embedder) are loaded."


# ✅ Launch the final clean LawBot UI
demo = gr.ChatInterface(
    fn=lawbot_chat,
    title="⚖️ LawBot — Indian Legal Assistant (Final RAG Version)",
    description="Ask about IPC, CrPC, or the Constitution — powered by RAG and fine-tuned Phi-3.",
    examples=[
        "What is the definition of theft under IPC?",
        "What is the punishment for robbery?",
        "Explain the scope of Section 144 of CrPC.",
        "What are the rights guaranteed under Article 21 of the Indian Constitution?"
    ]
)

# Launch with share=True to get a public link (lasts 72 hours)
demo.launch(share=True)

✅ Metadata re-loaded: 10279 entries.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cdf93f012f8baa8c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [55]:
# ============================================================
# ⚖️ LawBot — Indian Legal Assistant (Final RAG Version)
# ============================================================

import gradio as gr
import torch, textwrap, numpy as np, re, traceback, os, json

# --- RAG Components Reload (Safety Check) ---
try:
    meta_path = "/content/drive/MyDrive/LawBot_Project/LawBot_FAISS_Index/docs_fixed.jsonl"
    with open(meta_path, "r", encoding="utf-8") as f:
        metadata = [json.loads(line) for line in f]
    print(f"✅ Metadata re-loaded: {len(metadata)} entries.")
except Exception as e:
    print(f"❌ ERROR: Could not reload metadata.\nReason: {e}")

# Global chat history
chat_history = []

# ------------------------------------------------------------
# 🧠 LAWBot Core Chat Function
# ------------------------------------------------------------
def lawbot_chat(user_query, history):
    global chat_history
    try:
        # 1️⃣ Encode question & retrieve relevant chunks
        question_emb = embedder.encode([user_query])
        D, I = index.search(np.array(question_emb, dtype=np.float32), k=5)

        # 2️⃣ Retrieve top relevant chunks & sources
        context_chunks, sources = [], []
        for pos, idx in enumerate(I[0]):
            if 0 <= idx < len(metadata):
                entry = metadata[idx]
                text = entry.get("text", "")

                # Extract only "Answer" part for clarity
                ans_parts = re.split(r"(?i)\nA:\s*|\nAnswer\s*:\s*", text)
                clean_text = ans_parts[-1].strip() if len(ans_parts) > 1 else text

                context_chunks.append(clean_text)
                sources.append(f"{entry.get('source', 'unknown')} (score={D[0][pos]:.2f})")

        # Limit context for compactness
        context = "\n\n".join(context_chunks[:3])

        # 3️⃣ Build structured legal prompt
        history_text = "\n".join([f"User: {q}\nLawBot: {a}" for q, a in chat_history[-3:]])
        system_prompt = (
            "You are LawBot, a legal assistant trained on Indian law (IPC, CrPC, and the Constitution of India). "
            "Your task is to answer legal questions clearly and concisely using ONLY the provided Context. "
            "Always cite the relevant section number (e.g., 'Section 378 of IPC'). "
            "If information is unavailable in the context, say: 'Insufficient data in current legal database.' "
            "Do not repeat the prompt or previous conversation explicitly."
        )

        final_prompt = f"{system_prompt}\n\nPrevious Conversation:\n{history_text}\n\nContext (Legal References):\n{context}\n\nQuestion: {user_query}\n\nLawBot's Answer:"

        # 4️⃣ Generate output
        inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.4, top_p=0.9)
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # 5️⃣ Clean prompt bleed or repeated text
        clean_reply = reply
        if "LawBot's Answer:" in clean_reply:
            clean_reply = clean_reply.split("LawBot's Answer:")[-1].strip()

        clean_reply = re.sub(r"(?i)Question:.*?LawBot's Answer:", "", clean_reply, flags=re.DOTALL)
        clean_reply = re.sub(r"(?i)(you are lawbot.*?database\.)", "", clean_reply)
        clean_reply = re.sub(r"\s{2,}", " ", clean_reply).strip()

        # 6️⃣ Update history & format output
        chat_history.append((user_query, clean_reply))
        final_output = textwrap.fill(clean_reply, width=100)

        # 7️⃣ ✅ Source Display Logic (Improved Version)
        final_sources = set()
        for s in sources:
            clean_source = s.split("(")[0].strip()
            clean_source = os.path.basename(clean_source).replace(".jsonl", "").replace(".json", "")
            if clean_source and clean_source.lower() not in ["unknown", "none"]:
                final_sources.add(clean_source)

        if not final_sources:
            final_sources = {"lawbot_train", "ipc_qa"}  # fallback sources

        source_display_text = "\n".join(sorted(list(final_sources)))
        final_output += "\n\n📚 **Sources Referenced:**\n" + source_display_text

        return final_output

    except Exception as e:
        traceback.print_exc()
        return f"⚠️ Internal error: {str(e)}\nEnsure model, tokenizer, FAISS index, and embedder are loaded."


# ------------------------------------------------------------
# 🚀 Launch Gradio Chat UI
# ------------------------------------------------------------
demo = gr.ChatInterface(
    fn=lawbot_chat,
    title="⚖️ LawBot — Indian Legal Assistant",
    description="Ask about IPC, CrPC, or the Constitution — powered by RAG and fine-tuned Phi-3.",
    examples=[
        "What is the definition of theft under IPC?",
        "What is the punishment for robbery?",
        "Explain the scope of Section 144 of CrPC.",
        "What are the rights guaranteed under Article 21 of the Indian Constitution?"
    ]
)

demo.launch(share=True)


✅ Metadata re-loaded: 10279 entries.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://47c89b9d080f33cbba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
